In [1]:
!pip install diffusers transformers


In [2]:
import torch
import imageio
from diffusers import DDPMPipeline
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = DDPMPipeline.from_pretrained("google/ddpm-ema-celebahq-256")
pipe.to(device)

unet = pipe.unet
scheduler = pipe.scheduler

# Start from noise
x = torch.randn(1, 3, 256, 256, device=device)

scheduler.set_timesteps(200)   # use 200 steps for faster gif

frames = []

for t in scheduler.timesteps:
    with torch.no_grad():
        noise_pred = unet(x, t).sample

    x = scheduler.step(noise_pred, t, x).prev_sample

    # convert to image
    img = (x.clamp(-1, 1) + 1) / 2
    img = img.cpu().permute(0, 2, 3, 1).numpy()[0]

    # convert float [0,1] -> uint8 [0,255]
    img = (img * 255).astype(np.uint8)

    frames.append(img)

# save gif
imageio.mimsave("celeba_ddpm.gif", frames, fps=20, loop=0)

print("Saved gif as celeba_ddpm.gif")


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--google--ddpm-ema-celebahq-256/snapshots/4cb6117472e6e4f45c5afe606b101858c27c3802: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--google--ddpm-ema-celebahq-256/snapshots/4cb6117472e6e4f45c5afe606b101858c27c3802.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Saved gif as celeba_ddpm.gif
